# Ensemble Methods — Random Forests

This notebook demonstrates **bagging** and **random forests** using our NumPy-only implementations in `rice_ml.supervised_learning.ensemble_methods`.

We focus on two ideas:
- **Bagging reduces variance** by averaging many high-variance learners (decision trees).
- **Random feature subsampling** (`max_features`) decorrelates trees, improving the ensemble.

We use two classification datasets (Iris 2D and Breast Cancer) and two regression datasets (Diabetes and a 1D synthetic example).

## 1. Set-up

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

def add_repo_src_to_path(max_up: int = 8) -> None:
    cur = os.path.abspath(os.getcwd())
    for _ in range(max_up):
        candidate = os.path.join(cur, "src")
        if os.path.isdir(os.path.join(candidate, "rice_ml")):
            if candidate not in sys.path:
                sys.path.insert(0, candidate)
            return
        cur = os.path.abspath(os.path.join(cur, ".."))
    raise RuntimeError("Could not find 'src/rice_ml'. Run this notebook inside the repo, or install the package.")

add_repo_src_to_path()

from rice_ml.processing.preprocessing import standardize, train_test_split
from rice_ml.supervised_learning.decision_trees import DecisionTreeClassifier
from rice_ml.supervised_learning.regression_trees import RegressionTreeRegressor
from rice_ml.supervised_learning.ensemble_methods import RandomForestClassifier, RandomForestRegressor

from sklearn.datasets import load_iris, load_breast_cancer, load_diabetes

np.set_printoptions(precision=4, suppress=True)


## 2. Helper metrics + plotting

In [ ]:
def accuracy(y_true, y_pred) -> float:
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    return float(np.mean(y_true == y_pred))

def rmse(y_true, y_pred) -> float:
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

def r2_score(y_true, y_pred) -> float:
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    if ss_tot == 0:
        return 0.0
    return float(1.0 - ss_res / ss_tot)

def confusion_matrix_np(y_true, y_pred, labels=None):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    if labels is None:
        labels = np.unique(np.concatenate([y_true, y_pred]))
    labels = np.asarray(labels)
    k = labels.size
    cm = np.zeros((k, k), dtype=int)
    for i, a in enumerate(labels):
        for j, b in enumerate(labels):
            cm[i, j] = int(np.sum((y_true == a) & (y_pred == b)))
    return cm, labels

def plot_decision_regions_2d(
    X_raw,
    y,
    clf,
    title: str,
    xlabel: str,
    ylabel: str,
    mean=None,
    scale=None,
    h: float = 0.02,
):
    """Plot decision regions in the *raw* 2D feature space.

    If `mean` and `scale` are provided, the grid points are standardized
    before being passed to the classifier (useful when the model was fit
    on standardized features, but we want axes in original units).
    """
    X_raw = np.asarray(X_raw)
    y = np.asarray(y)

    x_min, x_max = X_raw[:, 0].min() - 0.6, X_raw[:, 0].max() + 0.6
    y_min, y_max = X_raw[:, 1].min() - 0.6, X_raw[:, 1].max() + 0.6

    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    grid_raw = np.c_[xx.ravel(), yy.ravel()]

    grid = grid_raw
    if mean is not None and scale is not None:
        grid = (grid_raw - mean) / scale

    Z = clf.predict(grid).reshape(xx.shape)

    plt.figure()
    plt.contourf(xx, yy, Z, alpha=0.25)
    plt.scatter(X_raw[:, 0], X_raw[:, 1], c=y, edgecolor="k", s=25)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()


## 3. Part A — Classification (Iris, 2D)

We use **petal length** and **petal width** (2D) to visualize decision boundaries.

In [ ]:
iris = load_iris()
X2 = iris.data[:, [2, 3]]  # petal length, petal width
y = iris.target

print("X2 shape:", X2.shape)
print("y shape :", y.shape)
print("classes :", list(enumerate(iris.target_names)))
unique, counts = np.unique(y, return_counts=True)
print("class counts:", dict(zip(unique, counts)))


In [ ]:
plt.figure()
plt.scatter(X2[:, 0], X2[:, 1], c=y, edgecolor="k", s=25)
plt.xlabel("petal length (cm)")
plt.ylabel("petal width (cm)")
plt.title("Iris (raw, 2D features)")
plt.show()


### 3.1 Train/test split + standardization

Trees/forests do **not** require feature scaling, but we keep a consistent preprocessing workflow and later verify that scaling does not change predictions (with fixed seeds).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X2, y,
    test_size=0.25,
    shuffle=True,
    stratify=y,
    random_state=42,
)

X_train_std, params = standardize(X_train, return_params=True)
mean = params["mean"]
scale = params["scale"]
X_test_std = (X_test - mean) / scale

print("Train mean (approx):", np.round(X_train_std.mean(axis=0), 6))
print("Train std  (approx):", np.round(X_train_std.std(axis=0), 6))
print("Test  mean (approx):", np.round(X_test_std.mean(axis=0), 6))
print("Test  std  (approx):", np.round(X_test_std.std(axis=0), 6))


### 3.2 Fit a Random Forest and visualize decision regions

In [ ]:
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=3,
    min_samples_leaf=1,
    max_features="sqrt",
    bootstrap=True,
    random_state=42,
).fit(X_train_std, y_train)

train_acc = accuracy(y_train, rf.predict(X_train_std))
test_acc = accuracy(y_test, rf.predict(X_test_std))

print("RandomForestClassifier (Iris 2D) | max_depth=3")
print("  train acc:", train_acc)
print("  test  acc:", test_acc)

y_pred = rf.predict(X_test_std)
cm, labels = confusion_matrix_np(y_test, y_pred, labels=[0, 1, 2])
print("\nConfusion matrix (test): rows=true, cols=pred, labels=[0,1,2]")
print(cm)


In [ ]:
plot_decision_regions_2d(
    X_train, y_train, rf,
    title="Decision regions (train) — Iris 2D | Random Forest | max_depth=3",
    xlabel="petal length (cm)",
    ylabel="petal width (cm)",
    mean=mean,
    scale=scale,
)


### 3.3 Scaling check: raw vs standardized

We fit the same forest twice (same `random_state`) on raw vs standardized features and compare predictions on the **test set**.

In [ ]:
rf_raw = RandomForestClassifier(
    n_estimators=200,
    max_depth=3,
    max_features="sqrt",
    bootstrap=True,
    random_state=42,
).fit(X_train, y_train)

rf_std = RandomForestClassifier(
    n_estimators=200,
    max_depth=3,
    max_features="sqrt",
    bootstrap=True,
    random_state=42,
).fit(X_train_std, y_train)

pred_raw = rf_raw.predict(X_test)
pred_std = rf_std.predict(X_test_std)

print("Test accuracy (raw features):", accuracy(y_test, pred_raw))
print("Test accuracy (standardized):", accuracy(y_test, pred_std))
print("Predictions identical?      :", bool(np.array_equal(pred_raw, pred_std)))


### 3.4 Hyperparameter sweeps (Iris 2D)

We scan `max_depth` and `n_estimators` to see the bias–variance pattern in classification.

In [ ]:
depths = list(range(1, 11))
train_accs = []
test_accs = []

for d in depths:
    clf = RandomForestClassifier(
        n_estimators=200,
        max_depth=d,
        max_features="sqrt",
        bootstrap=True,
        random_state=42,
    ).fit(X_train_std, y_train)
    train_accs.append(accuracy(y_train, clf.predict(X_train_std)))
    test_accs.append(accuracy(y_test, clf.predict(X_test_std)))

best_d = depths[int(np.argmax(test_accs))]

plt.figure()
plt.plot(depths, train_accs, marker="o", label="train accuracy")
plt.plot(depths, test_accs, marker="o", label="test accuracy")
plt.axvline(best_d, linestyle="--", label=f"best depth = {best_d}")
plt.xlabel("max_depth")
plt.ylabel("accuracy")
plt.title("Iris 2D — Accuracy vs max_depth (Random Forest)")
plt.legend()
plt.show()

print("Best depth by test accuracy:", best_d)


In [ ]:
n_list = [1, 5, 10, 25, 50, 100, 200, 400]
train_accs_n = []
test_accs_n = []

for n_estimators in n_list:
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=best_d,
        max_features="sqrt",
        bootstrap=True,
        random_state=42,
    ).fit(X_train_std, y_train)
    train_accs_n.append(accuracy(y_train, clf.predict(X_train_std)))
    test_accs_n.append(accuracy(y_test, clf.predict(X_test_std)))

plt.figure()
plt.plot(n_list, train_accs_n, marker="o", label="train accuracy")
plt.plot(n_list, test_accs_n, marker="o", label="test accuracy")
plt.xlabel("n_estimators")
plt.ylabel("accuracy")
plt.title(f"Iris 2D — Accuracy vs n_estimators (max_depth={best_d})")
plt.legend()
plt.show()


In [ ]:
settings = [
    ("all (None)", None),
    ("sqrt(p)", "sqrt"),
    ("0.3*p", 0.3),
]

rows = []
for name, mf in settings:
    clf = RandomForestClassifier(
        n_estimators=200,
        max_depth=best_d,
        max_features=mf,
        bootstrap=True,
        random_state=42,
    ).fit(X_train_std, y_train)
    rows.append([
        name,
        mf,
        accuracy(y_train, clf.predict(X_train_std)),
        accuracy(y_test, clf.predict(X_test_std)),
    ])

print("setting\tmax_features\ttrain_acc\ttest_acc")
for r in rows:
    print(f"{r[0]}\t{r[1]}\t{r[2]:.4f}\t{r[3]:.4f}")


## 4. Part B — Classification (Breast Cancer, 30D)

We compare a **single decision tree** vs a **random forest** on a higher-dimensional dataset.

In [ ]:
data = load_breast_cancer()
X = data.data
y = data.target  # {0,1}

print("X shape:", X.shape)
print("y shape:", y.shape)
print("classes:", list(enumerate(data.target_names)))
unique, counts = np.unique(y, return_counts=True)
print("class counts:", dict(zip(unique, counts)))
print("Any NaN in X?", bool(np.isnan(X).any()))


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    shuffle=True,
    stratify=y,
    random_state=42,
)

X_train_std, params = standardize(X_train, return_params=True)
mean = params["mean"]
scale = params["scale"]
X_test_std = (X_test - mean) / scale


### 4.1 Single tree vs forest (same depth)

A forest should reduce variance relative to a single high-variance tree.

In [ ]:
tree = DecisionTreeClassifier(max_depth=5, random_state=42).fit(X_train_std, y_train)
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=5,
    max_features="sqrt",
    random_state=42,
).fit(X_train_std, y_train)

print("DecisionTreeClassifier | max_depth=5")
print("  train acc:", accuracy(y_train, tree.predict(X_train_std)))
print("  test  acc:", accuracy(y_test, tree.predict(X_test_std)))

print("\nRandomForestClassifier | n_estimators=300, max_depth=5, max_features='sqrt'")
print("  train acc:", accuracy(y_train, rf.predict(X_train_std)))
print("  test  acc:", accuracy(y_test, rf.predict(X_test_std)))


### 4.2 Sweep `max_depth` (forest)

We scan depth and see where test accuracy stabilizes.

In [ ]:
depths = list(range(1, 16))
train_accs = []
test_accs = []

for d in depths:
    clf = RandomForestClassifier(
        n_estimators=300,
        max_depth=d,
        max_features="sqrt",
        random_state=42,
    ).fit(X_train_std, y_train)
    train_accs.append(accuracy(y_train, clf.predict(X_train_std)))
    test_accs.append(accuracy(y_test, clf.predict(X_test_std)))

best_d_bc = depths[int(np.argmax(test_accs))]

plt.figure()
plt.plot(depths, train_accs, marker="o", label="train accuracy")
plt.plot(depths, test_accs, marker="o", label="test accuracy")
plt.axvline(best_d_bc, linestyle="--", label=f"best depth = {best_d_bc}")
plt.xlabel("max_depth")
plt.ylabel("accuracy")
plt.title("Breast Cancer — Accuracy vs max_depth (Random Forest)")
plt.legend()
plt.show()

print("Best depth by test accuracy:", best_d_bc)


### 4.3 Sweep `n_estimators` (forest)

More trees typically reduce variance and stabilize test performance.

In [ ]:
n_list = [1, 5, 10, 25, 50, 100, 200, 400]
train_accs_n = []
test_accs_n = []

for n_estimators in n_list:
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=best_d_bc,
        max_features="sqrt",
        random_state=42,
    ).fit(X_train_std, y_train)
    train_accs_n.append(accuracy(y_train, clf.predict(X_train_std)))
    test_accs_n.append(accuracy(y_test, clf.predict(X_test_std)))

plt.figure()
plt.plot(n_list, train_accs_n, marker="o", label="train accuracy")
plt.plot(n_list, test_accs_n, marker="o", label="test accuracy")
plt.xlabel("n_estimators")
plt.ylabel("accuracy")
plt.title(f"Breast Cancer — Accuracy vs n_estimators (max_depth={best_d_bc})")
plt.legend()
plt.show()


### 4.4 Compare `max_features` settings (forest)

Forests often benefit from restricting features per split (decorrelation).

In [ ]:
settings = [
    ("all (None)", None),
    ("sqrt(p)", "sqrt"),
    ("log2(p)", "log2"),
    ("0.3*p", 0.3),
]

rows = []
for name, mf in settings:
    clf = RandomForestClassifier(
        n_estimators=300,
        max_depth=best_d_bc,
        max_features=mf,
        random_state=42,
    ).fit(X_train_std, y_train)
    rows.append([
        name,
        mf,
        accuracy(y_train, clf.predict(X_train_std)),
        accuracy(y_test, clf.predict(X_test_std)),
    ])

print("setting\tmax_features\ttrain_acc\ttest_acc")
for r in rows:
    print(f"{r[0]}\t{r[1]}\t{r[2]:.4f}\t{r[3]:.4f}")


## 5. Part C — Regression (Diabetes)

We compare a single regression tree vs a random forest regressor using **RMSE** and **R²**.

To keep runtimes reasonable and match earlier notebooks, we use **5 features** (the first 5).

In [ ]:
diab = load_diabetes()
X = diab.data[:, :5]  # 5 features
y = diab.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    shuffle=True,
    random_state=42,
)

X_train_std, params = standardize(X_train, return_params=True)
mean = params["mean"]
scale = params["scale"]
X_test_std = (X_test - mean) / scale

print("Train mean (approx):", np.round(X_train_std.mean(axis=0), 4))
print("Train std  (approx):", np.round(X_train_std.std(axis=0), 4))
print("Test  mean (approx):", np.round(X_test_std.mean(axis=0), 4))
print("Test  std  (approx):", np.round(X_test_std.std(axis=0), 4))


### 5.1 Baselines: mean predictor, single tree, random forest

In [ ]:
# Mean baseline (predict training mean)
baseline_pred = np.full_like(y_test, float(np.mean(y_train)))
print("Mean baseline")
print("  test  RMSE:", rmse(y_test, baseline_pred))
print("  test  R^2 :", r2_score(y_test, baseline_pred))

# Single regression tree
tree = RegressionTreeRegressor(max_depth=5, min_samples_leaf=5, random_state=42).fit(X_train_std, y_train)
pred_tree_train = tree.predict(X_train_std)
pred_tree_test = tree.predict(X_test_std)

print("\nRegressionTreeRegressor | max_depth=5, min_samples_leaf=5")
print("  train RMSE:", rmse(y_train, pred_tree_train))
print("  test  RMSE:", rmse(y_test, pred_tree_test))
print("  train R^2 :", r2_score(y_train, pred_tree_train))
print("  test  R^2 :", r2_score(y_test, pred_tree_test))

# Random forest regressor
rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=5,
    min_samples_leaf=5,
    max_features="sqrt",
    random_state=42,
).fit(X_train_std, y_train)

pred_rf_train = rf.predict(X_train_std)
pred_rf_test = rf.predict(X_test_std)

print("\nRandomForestRegressor | n_estimators=300, max_depth=5, min_samples_leaf=5")
print("  train RMSE:", rmse(y_train, pred_rf_train))
print("  test  RMSE:", rmse(y_test, pred_rf_test))
print("  train R^2 :", r2_score(y_train, pred_rf_train))
print("  test  R^2 :", r2_score(y_test, pred_rf_test))


### 5.2 Sweep `max_depth` (forest)

We choose `best_depth` by **minimum test RMSE**.

In [ ]:
depths = list(range(1, 21))
train_rmse = []
test_rmse = []
train_r2 = []
test_r2 = []

for d in depths:
    model = RandomForestRegressor(
        n_estimators=300,
        max_depth=d,
        min_samples_leaf=5,
        max_features="sqrt",
        random_state=42,
    ).fit(X_train_std, y_train)

    pr_tr = model.predict(X_train_std)
    pr_te = model.predict(X_test_std)

    train_rmse.append(rmse(y_train, pr_tr))
    test_rmse.append(rmse(y_test, pr_te))
    train_r2.append(r2_score(y_train, pr_tr))
    test_r2.append(r2_score(y_test, pr_te))

best_d = depths[int(np.argmin(test_rmse))]

plt.figure()
plt.plot(depths, train_rmse, marker="o", label="train RMSE")
plt.plot(depths, test_rmse, marker="o", label="test RMSE")
plt.axvline(best_d, linestyle="--", label=f"best depth = {best_d}")
plt.xlabel("max_depth")
plt.ylabel("RMSE")
plt.title("Diabetes — RMSE vs max_depth (Random Forest)")
plt.legend()
plt.show()

plt.figure()
plt.plot(depths, train_r2, marker="o", label="train R^2")
plt.plot(depths, test_r2, marker="o", label="test R^2")
plt.axvline(best_d, linestyle="--", label=f"best depth = {best_d}")
plt.xlabel("max_depth")
plt.ylabel("R^2")
plt.title("Diabetes — R^2 vs max_depth (Random Forest)")
plt.legend()
plt.show()

print("Best depth by test RMSE:", best_d)


### 5.3 Sweep `min_samples_leaf` (forest, fixed depth)

Larger leaf sizes regularize the forest and can improve generalization on noisy data.

In [ ]:
leaves = [1, 2, 5, 10, 20, 40, 80]
train_rmse_l = []
test_rmse_l = []
train_r2_l = []
test_r2_l = []

for leaf in leaves:
    model = RandomForestRegressor(
        n_estimators=300,
        max_depth=best_d,
        min_samples_leaf=leaf,
        max_features="sqrt",
        random_state=42,
    ).fit(X_train_std, y_train)

    pr_tr = model.predict(X_train_std)
    pr_te = model.predict(X_test_std)

    train_rmse_l.append(rmse(y_train, pr_tr))
    test_rmse_l.append(rmse(y_test, pr_te))
    train_r2_l.append(r2_score(y_train, pr_tr))
    test_r2_l.append(r2_score(y_test, pr_te))

best_leaf = leaves[int(np.argmin(test_rmse_l))]

plt.figure()
plt.plot(leaves, train_rmse_l, marker="o", label="train RMSE")
plt.plot(leaves, test_rmse_l, marker="o", label="test RMSE")
plt.axvline(best_leaf, linestyle="--", label=f"best leaf = {best_leaf}")
plt.xlabel("min_samples_leaf")
plt.ylabel("RMSE")
plt.title(f"Diabetes — RMSE vs min_samples_leaf (max_depth={best_d})")
plt.legend()
plt.show()

plt.figure()
plt.plot(leaves, train_r2_l, marker="o", label="train R^2")
plt.plot(leaves, test_r2_l, marker="o", label="test R^2")
plt.axvline(best_leaf, linestyle="--", label=f"best leaf = {best_leaf}")
plt.xlabel("min_samples_leaf")
plt.ylabel("R^2")
plt.title(f"Diabetes — R^2 vs min_samples_leaf (max_depth={best_d})")
plt.legend()
plt.show()

print("Best min_samples_leaf (with best_depth):", best_leaf)


### 5.4 Compare `max_features` settings (forest)

For regression forests, feature subsampling can still help by decorrelating trees, but the best choice is dataset-dependent.

In [ ]:
settings = [
    ("all (None)", None),
    ("sqrt(p)", "sqrt"),
    ("0.3*p", 0.3),
]

rows = []
for name, mf in settings:
    model = RandomForestRegressor(
        n_estimators=300,
        max_depth=best_d,
        min_samples_leaf=best_leaf,
        max_features=mf,
        random_state=42,
    ).fit(X_train_std, y_train)

    pr_tr = model.predict(X_train_std)
    pr_te = model.predict(X_test_std)

    rows.append([
        name,
        mf,
        rmse(y_train, pr_tr),
        rmse(y_test, pr_te),
        r2_score(y_train, pr_tr),
        r2_score(y_test, pr_te),
    ])

print("setting\tmax_features\ttrain_RMSE\ttest_RMSE\ttrain_R2\ttest_R2")
for r in rows:
    print(f"{r[0]}\t{r[1]}\t{r[2]:.4f}\t{r[3]:.4f}\t{r[4]:.4f}\t{r[5]:.4f}")


## 6. Part D — 1D synthetic regression (visual intuition)

Regression forests average many piecewise-constant trees, which often produces a **less jagged** function than a single deep tree.

In [ ]:
rng = np.random.default_rng(0)
n = 300
x = rng.uniform(-3.0, 3.0, size=n)
y = np.sin(x) + 0.15 * rng.normal(size=n)

X1 = x.reshape(-1, 1)

X1_train, X1_test, y1_train, y1_test = train_test_split(
    X1, y,
    test_size=0.25,
    shuffle=True,
    random_state=42,
)

X1_train_std, params1 = standardize(X1_train, return_params=True)
mean1 = params1["mean"]
scale1 = params1["scale"]
X1_test_std = (X1_test - mean1) / scale1

print("X1_train shape:", X1_train.shape, "X1_test shape:", X1_test.shape)
print("Train mean (approx):", np.round(X1_train_std.mean(axis=0), 6))
print("Train std  (approx):", np.round(X1_train_std.std(axis=0), 6))


In [ ]:
grid = np.linspace(X1_train_std.min() - 0.5, X1_train_std.max() + 0.5, 600).reshape(-1, 1)

plt.figure()
plt.scatter(X1_train_std[:, 0], y1_train, s=15, label="train")
plt.scatter(X1_test_std[:, 0], y1_test, s=15, label="test")

configs = [
    ("n=5", 5),
    ("n=25", 25),
    ("n=200", 200),
]
for label, n_estimators in configs:
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=8,
        min_samples_leaf=2,
        max_features=None,  # 1D
        random_state=0,
    ).fit(X1_train_std, y1_train)
    plt.plot(grid[:, 0], model.predict(grid), label=label)

plt.xlabel("x (standardized)")
plt.ylabel("y")
plt.title("1D Synthetic — Random forest predictions (averaging piecewise-constant trees)")
plt.legend()
plt.show()


In [ ]:
depths = list(range(1, 21))
test_rmse = []
train_rmse = []

for d in depths:
    model = RandomForestRegressor(
        n_estimators=200,
        max_depth=d,
        min_samples_leaf=2,
        max_features=None,
        random_state=0,
    ).fit(X1_train_std, y1_train)

    train_rmse.append(rmse(y1_train, model.predict(X1_train_std)))
    test_rmse.append(rmse(y1_test, model.predict(X1_test_std)))

best_d_syn = depths[int(np.argmin(test_rmse))]

plt.figure()
plt.plot(depths, train_rmse, marker="o", label="train RMSE")
plt.plot(depths, test_rmse, marker="o", label="test RMSE")
plt.axvline(best_d_syn, linestyle="--", label=f"best depth = {best_d_syn}")
plt.xlabel("max_depth")
plt.ylabel("RMSE")
plt.title("1D Synthetic — RMSE vs max_depth (Random Forest)")
plt.legend()
plt.show()

print("Best depth by test RMSE (synthetic):", best_d_syn)


## 7. Conclusion

- **Bagging** (bootstrap + averaging) reduces variance: forests typically generalize better than a single deep tree.
- Increasing **tree depth** drives training accuracy/R² up, but test performance often peaks at a moderate depth.
- Increasing **`n_estimators`** usually stabilizes test performance (lower variance), with diminishing returns.
- **Feature subsampling** (`max_features='sqrt'` / `'log2'` / fractions) is a key ingredient for random forests because it decorrelates trees.
- Even though trees don’t require scaling, keeping a consistent **train-only preprocessing** step helps avoid leakage and makes experiments comparable across models.